# Text Classification (UTS)

## Data Preparation

download requisite libraries and packages

In [68]:
# Sentence Transformers: Multilingual Sentence, Paragraph, and Image Embeddings using BERT & Co.
!pip install sentence-transformers
!pip install git+https://github.com/MartinoMensio/spacy-sentence-bert.git

# for model paraphrase-xlm-r-multilingual-v1
!pip install https://github.com/MartinoMensio/spacy-sentence-bert/releases/download/v0.1.2/xx_paraphrase_xlm_r_multilingual_v1-0.1.2.tar.gz # xx_paraphrase_xlm_r_multilingual_v1-0.1.2

# for model distiluse-base-multilingual-cased-v2
!pip install https://github.com/MartinoMensio/spacy-sentence-bert/releases/download/v0.1.2/xx_distiluse_base_multilingual_cased_v2-0.1.2.tar.gz #xx_distiluse_base_multilingual_cased_v2-0.1.2

# Stem for Indonesian Language
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MartinoMensio/spacy-sentence-bert.git to /tmp/pip-req-build-ng9zzaoz
  Running command git clone --filter=blob:none --quiet https://github.com/MartinoMensio/spacy-sentence-bert.git /tmp/pip-req-build-ng9zzaoz
  Resolved https://github.com/MartinoMensio/spacy-sentence-bert.git to commit 9fb25f92b5be77d42921a66fda2c54caa3199925
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for xx-distiluse-base-multilingual-cased-v2: filename=xx_distiluse_base_multilingual_cased_v2-0.1.2-py3-none

Import requisite libraries

In [69]:
import pandas as pd


#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# stemming for Indonesian Language
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('punkt')
nltk.download('stopwords')


#for model-building
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


#for word embedding
import gensim
from gensim.models import Word2Vec

# for NLP use BERT
from sentence_transformers import SentenceTransformer
import spacy_sentence_bert

import warnings
warnings.filterwarnings(action = 'ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Dataset preparation

In [76]:
df = pd.read_csv("/content/drive/MyDrive/prosaindata/source/tasks/output/Data_TA.csv")
df = df.dropna()
df = df.reset_index(drop=True)

In [77]:
df.head(10)

,NPM,Judul,Abstrak,Prodi,Label
0,40411100468,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE T...,Sistem informasi akademik (SIAKAD) merupaka...,Teknik Informatika,RPL
1,40411100476,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,Berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika,RPL
2,40411100480,RANCANG BANGUN APLIKASI PROXY SERVER UNTUKENKR...,Web server adalah sebuah perangkat lunak serve...,Teknik Informatika,RPL
3,70411100070,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Penjadwalan kuliah di Perguruan Tinggi me...,Teknik Informatika,komputasi
4,80411100115,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Seiring perkembangan teknologi yang ada diduni...,Teknik Informatika,komputasi
5,70411100007,Gerak Pekerja Pada Game Real Time Strategy Men...,Gerak pekerja ada pada game yang memiliki genr...,Teknik Informatika,komputasi
6,70411100126,RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MEN...,"Perkembangan game yang semakin pesat, memberik...",Teknik Informatika,komputasi
7,70411100109,EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEA...,Sistem pengenalan wajah adalah suatu sistem un...,Teknik Informatika,komputasi
8,80411100083,IMPLEMENTASI ALGORITMA PRIM DAN DEPTH FIRST ...,Teknologi mobile game beroperating system open...,Teknik Informatika,komputasi
9,70411100092,Perancangan Sistem Informasi Badan Kepegawaian...,Kantor Badan Kepegawaian kota Bangkalan adalah...,Teknik Informatika,RPL


In [78]:
label = df["Label"]

In [79]:
abstract_datas = df["Abstrak"]

In [80]:
abstract_datas.head(10)

0    Sistem  informasi  akademik  (SIAKAD) merupaka...
1    Berjalannya koneksi jaringan komputer dengan l...
2    Web server adalah sebuah perangkat lunak serve...
3    Penjadwalan  kuliah  di  Perguruan  Tinggi  me...
4    Seiring perkembangan teknologi yang ada diduni...
5    Gerak pekerja ada pada game yang memiliki genr...
6    Perkembangan game yang semakin pesat, memberik...
7    Sistem pengenalan wajah adalah suatu sistem un...
8    Teknologi mobile game beroperating system open...
9    Kantor Badan Kepegawaian kota Bangkalan adalah...
Name: Abstrak, dtype: object

## Preprocessing

In [81]:
def preprocess(text):
    """convert to lowercase, strip and remove punctuations"""
    text = text.lower() 

    # remove whitespace
    text = text.strip() 
    # compile pattern that have `<.*?>` then replace with ''
    text = re.compile('<.*?>').sub('', text) 

    # compile pattern that have punctuation !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~ then replace with ' '
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)

    # replace all one or more space to one space
    text = re.sub('\s+', ' ', text)

    # replace all number 0-9 to one space
    text = re.sub(r"\d+", "", text)

    # replace all first word or space in the beginning of line to ''
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    # replace all digits to one space
    text = re.sub(r'\d',' ',text) 

    # replace all one or more space to one space
    text = re.sub(r'\s+',' ',text) 

    return text

def stopword(string):
    """Stopword removal like I, The, An, be, is, etc"""
    a = [i for i in string.split() if i not in stopwords.words('indonesian')]
    return ' '.join(a)

Clean text using ```preprocess()``` function from number, capitalize, punctuation, and other unnecessary characters

In [82]:
preprocessed_abstract = []
for text in abstract_datas:
    preprocessed_abstract.append(preprocess(str(text)))

In [83]:
preprocessed_abstract = pd.Series(preprocessed_abstract)

In [84]:
preprocessed_abstract[90]

'e learning merupakan usaha untuk membuat sebuah transformasi proses belajar mengajar di sekolah dalam bentuk digital yang dijembatani oleh teknologi internet sudah banyak pembelajaran saat ini yang telah didukung oleh e learning namun dalam proses pembelajaran setiap peserta didik mempunyai gaya belajar yang berbeda satu sama lain gaya pembelajaran yang berbeda tersebut dapat dikelompokkan dengan cara menggunakan tool kuisioner dan secara otomatis penelitian ini akan membahas bagaimana menganalisis gaya belajar peserta didik dan mengelompokkannya dengan menggunakan tool kuisioner serta mengklasifikasi gaya belajar peserta didik menggunkan naãve bayes classifier dimensi untuk model gaya belajar yang digunakan adalah felder silvermen learning style model fslsm dengan penelitian ini diharapkan agar dapat mengatasi permasalahan keragaman gaya belajar peserta didik sehingga dapat meningkatkan efektivitas pembelajaran e learning'

Clean text use ```stopword()``` function from conjuction and other unnecessary words 

In [85]:
removed_stopword_abstract = []
for text in preprocessed_abstract:
    removed_stopword_abstract.append(stopword(str(text)))

In [86]:
removed_stopword_abstract = pd.Series(removed_stopword_abstract)

In [87]:
removed_stopword_abstract[90]

'e learning usaha transformasi proses belajar mengajar sekolah bentuk digital dijembatani teknologi internet pembelajaran didukung e learning proses pembelajaran peserta didik gaya belajar berbeda gaya pembelajaran berbeda dikelompokkan tool kuisioner otomatis penelitian membahas menganalisis gaya belajar peserta didik mengelompokkannya tool kuisioner mengklasifikasi gaya belajar peserta didik menggunkan naãve bayes classifier dimensi model gaya belajar felder silvermen learning style model fslsm penelitian diharapkan mengatasi permasalahan keragaman gaya belajar peserta didik meningkatkan efektivitas pembelajaran e learning'

Clean text use ```to_base_word()``` function to make all the words root words

In [88]:
# sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def to_base_word(string):
    """Change words to base words"""
    string = string.split()
    text = [stemmer.stem(word) for word in string]
    return ' '.join(text)

In [89]:
cleaned_abstract = []
for text in removed_stopword_abstract:
    cleaned_abstract.append(to_base_word(str(text)))

In [90]:
cleaned_abstract = pd.Series(cleaned_abstract)

In [91]:
cleaned_abstract[90]

'e learning usaha transformasi proses ajar ajar sekolah bentuk digital jembatan teknologi internet ajar dukung e learning proses ajar serta didik gaya ajar beda gaya ajar beda kelompok tool kuisioner otomatis teliti bahas analis gaya ajar serta didik kelompok tool kuisioner klasifikasi gaya ajar serta didik gun na ve bayes classifier dimensi model gaya ajar felder silvermen learning style model fslsm teliti harap atas masalah ragam gaya ajar serta didik tingkat efektivitas ajar e learning'

## Word Embedding

```Cosine Simmilarity``` 


Diukur dengan cosinus sudut antara dua vektor dan menentukan apakah dua vektor menunjuk ke arah yang kira-kira sama.

```vector_size (int, optional)``` – Dimensionality of the word vectors.


```window (int, optional)``` – Maximum distance between the current and predicted word within a sentence.


```min_count (int, optional)``` – Ignores all words with a lower total frequency than this.

### Word embedding use CBOW and Skip Gram model

In [92]:
# Python program to generate word vectors using Word2Vec
data = []
# iterate through each sentence in the file
for text in cleaned_abstract:
    for i in sent_tokenize(text):
        temp = []
        
        # tokenize the sentence into words
        for j in word_tokenize(i):
            temp.append(j.lower())
    
        data.append(temp)

In [93]:
len(data)

810

dummy data for test CBOW and Skip Gram Model

In [94]:
first_word1 = "sistem"
first_word2 = "game"
first_word3 = "komputasi"

second_word1 = "perangkat"
second_word2 = "perangkat"
second_word3 = "perangkat"

#### Continous Bag of Word Model

In [95]:
# Create CBOW model
cbow_model = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5)

In [96]:
result_with_cbow = cbow_model.wv.similarity(first_word1, second_word1)
print(f"Cosine similarity between '{first_word1}' and '{second_word1}' - with CBOW : {result_with_cbow}")

Cosine similarity between 'sistem' and 'perangkat' - with CBOW : 0.9801543354988098


In [97]:
result_with_cbow = cbow_model.wv.similarity(first_word2, second_word2)
print(f"Cosine similarity between '{first_word2}' and '{second_word2}' - with CBOW : {result_with_cbow}")

Cosine similarity between 'game' and 'perangkat' - with CBOW : 0.9825574159622192


In [98]:
result_with_cbow = cbow_model.wv.similarity(first_word3, second_word3)
print(f"Cosine similarity between '{first_word3}' and '{second_word3}' - with CBOW : {result_with_cbow}")

Cosine similarity between 'komputasi' and 'perangkat' - with CBOW : 0.9897515177726746


#### Skip Gram Model

In [99]:
# Create Skip Gram model
skip_gram_model = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100,window = 5, sg = 1)

In [100]:
result_with_cbow = skip_gram_model.wv.similarity(first_word1, second_word1)
print(f"Cosine similarity between '{first_word1}' and '{second_word1}' - with CBOW : {result_with_cbow}")

Cosine similarity between 'sistem' and 'perangkat' - with CBOW : 0.7361634969711304


In [101]:
result_with_cbow = skip_gram_model.wv.similarity(first_word2, second_word2)
print(f"Cosine similarity between '{first_word2}' and '{second_word2}' - with CBOW : {result_with_cbow}")

Cosine similarity between 'game' and 'perangkat' - with CBOW : 0.7570070028305054


In [102]:
result_with_cbow = skip_gram_model.wv.similarity(first_word3, second_word3)
print(f"Cosine similarity between '{first_word3}' and '{second_word3}' - with CBOW : {result_with_cbow}")

Cosine similarity between 'komputasi' and 'perangkat' - with CBOW : 0.6018103361129761


## Evaluation

### Evaluation Preparation

Show all available labels as ```y```

In [103]:
print(label.unique())

['RPL' 'komputasi' 'PBA']


Overview of abstracr as ```X```

In [104]:
cleaned_abstract

0      sistem informasi akademik siakad sistem inform...
1      jalan koneksi jaring komputer lancar ganggu ha...
2      web server perangkat lunak server berfungsimen...
3      jadwal kuliah guru kompleks masalah variabel t...
4      iring kembang teknologi dunia muncul teknologi...
                             ...                        
805    investasi saham milik resiko rugi dikarenakanp...
806    information retrieval ir ambil informasi simpa...
807    klasifikasi citra proses kelompok piksel citra...
808    identifikasi atribut pejal kaki salah teliti k...
809    topik deteksi objek tarik perhati kembang tekn...
Length: 810, dtype: object

create new normalised and cleaned data frame from raw data

In [105]:
new_data = {"Abstract" : cleaned_abstract, "Label" : label}
new_df = pd.concat(new_data, axis = 1)

In [106]:
new_df

,Abstract,Label
0,sistem informasi akademik siakad sistem inform...,RPL
1,jalan koneksi jaring komputer lancar ganggu ha...,RPL
2,web server perangkat lunak server berfungsimen...,RPL
3,jadwal kuliah guru kompleks masalah variabel t...,komputasi
4,iring kembang teknologi dunia muncul teknologi...,komputasi
...,...,...
805,investasi saham milik resiko rugi dikarenakanp...,komputasi
806,information retrieval ir ambil informasi simpa...,PBA
807,klasifikasi citra proses kelompok piksel citra...,komputasi
808,identifikasi atribut pejal kaki salah teliti k...,komputasi


load one of the models listed at https://github.com/MartinoMensio/spacy-sentence-bert/

In [107]:
# for model paraphrase-xlm-r-multilingual-v1
# nlp = spacy_sentence_bert.load_model('xx_paraphrase_xlm_r_multilingual_v1')

# for model distiluse-base-multilingual-cased-v2
nlp = spacy_sentence_bert.load_model('xx_distiluse_base_multilingual_cased_v2')

create ```df['vector']``` where individual abstract are represented as real-valued vectors in a lower-dimensional space

In [108]:
new_df['vector'] = new_df['Abstract'].apply(lambda x: nlp(x).vector)

In [109]:
new_df

,Abstract,Label,vector
0,sistem informasi akademik siakad sistem inform...,RPL,"[0.031695776, 0.0023456744, 0.024228357, -0.02..."
1,jalan koneksi jaring komputer lancar ganggu ha...,RPL,"[0.05129607, -0.024710627, 0.007151665, 0.0053..."
2,web server perangkat lunak server berfungsimen...,RPL,"[-0.018094577, 0.008628332, 0.018115591, -0.04..."
3,jadwal kuliah guru kompleks masalah variabel t...,komputasi,"[0.022376861, 0.04639912, 0.023099946, -0.0318..."
4,iring kembang teknologi dunia muncul teknologi...,komputasi,"[0.01736855, 0.012109187, 0.0017237916, -0.039..."
...,...,...,...
805,investasi saham milik resiko rugi dikarenakanp...,komputasi,"[0.069311336, -0.04580292, -0.037823424, -0.02..."
806,information retrieval ir ambil informasi simpa...,PBA,"[0.018961675, 0.058011264, 0.00534609, -0.0410..."
807,klasifikasi citra proses kelompok piksel citra...,komputasi,"[-0.0014633922, 0.0130769415, 0.013453827, -0...."
808,identifikasi atribut pejal kaki salah teliti k...,komputasi,"[0.00845348, 0.016413385, 0.040197726, -0.0150..."


Split X and y as below


*   ```X_train``` train X data 
*   ```X_test``` test X data 


*   ```y_train``` train y data 
*   ```y_test``` test y data 



In [110]:
X_train, X_test, y_train, y_test = train_test_split(new_df['vector'].tolist(), new_df['Label'].tolist(), test_size=0.33, random_state=42)

### Train with machine learning classifier.

#### Support Vector Classifier(SVM)

In [111]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(gamma='auto')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.7126865671641791


#### Random Forest Classifier

In [112]:
clf = RandomForestClassifier(max_depth=9, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.7611940298507462


### Classification Test

Given dummy data to classify 


*   ```abstracts``` as data 
*   ```labels``` as category



In [113]:
abstracts = [
    "Sistem  informasi  akademik  (SIAKAD) merupakan  sistem  informasi  yang  berfungsi  menangani pengelolaan  dan  penyajian  data-data  akademik,  yang  oleh pihak  fakultas  SIAKAD  dianggap  sangat  penting  dalam memberikan  pelayanan  mahasiswa  yang  membutuhkan informasi akademik.",
    "Web server adalah sebuah perangkat lunak server yang berfungsimenerima permintaan HTTP atau HTTPS dari web client danmengirimkan jawaban dalam bentuk halaman-halaman web yangumumnya berbentuk dokumen HTML. Dokumen HTML yang dikirimmerupakan dokumen yang dibentuk dari bahasa markup yang plaintext.Artinya, dokumen berisi kode-kode HTML murni tanpa proses enkripsi.",
    "Berjalannya koneksi jaringan komputer dengan lancar dan tanpa gangguan merupakan harapan seluruh penggunanya,terutama pihak yang bertanggung jawab pada bidang itu, yakni administrator jaringan.",
]

labels = [
    'RPL',  
    'RPL',  
    'RPL'
]

Result


result when use model paraphrase-xlm-r-multilingual-v1

In [ ]:
# for abstract, label in zip(abstracts, labels):
#   print(abstract)
#   print(f"True Label: {label}, Predicted Label: {clf.predict(nlp(abstract).vector.reshape(1, -1))[0]} \n")

# OUPUT
# Sistem  informasi  akademik  (SIAKAD) merupakan  sistem  informasi  yang  berfungsi  menangani pengelolaan  dan  penyajian  data-data  akademik,  yang  oleh pihak  fakultas  SIAKAD  dianggap  sangat  penting  dalam memberikan  pelayanan  mahasiswa  yang  membutuhkan informasi akademik.
# True Label: RPL, Predicted Label: komputasi 

# Web server adalah sebuah perangkat lunak server yang berfungsimenerima permintaan HTTP atau HTTPS dari web client danmengirimkan jawaban dalam bentuk halaman-halaman web yangumumnya berbentuk dokumen HTML. Dokumen HTML yang dikirimmerupakan dokumen yang dibentuk dari bahasa markup yang plaintext.Artinya, dokumen berisi kode-kode HTML murni tanpa proses enkripsi.
# True Label: RPL, Predicted Label: komputasi 

# Berjalannya koneksi jaringan komputer dengan lancar dan tanpa gangguan merupakan harapan seluruh penggunanya,terutama pihak yang bertanggung jawab pada bidang itu, yakni administrator jaringan.
# True Label: RPL, Predicted Label: komputasi 


In [115]:
for abstract, label in zip(abstracts, labels):
  print(abstract)
  print(f"True Label: {label}, Predicted Label: {clf.predict(nlp(abstract).vector.reshape(1, -1))[0]} \n")

# OUPUT
# Sistem  informasi  akademik  (SIAKAD) merupakan  sistem  informasi  yang  berfungsi  menangani pengelolaan  dan  penyajian  data-data  akademik,  yang  oleh pihak  fakultas  SIAKAD  dianggap  sangat  penting  dalam memberikan  pelayanan  mahasiswa  yang  membutuhkan informasi akademik.
# True Label: RPL, Predicted Label: komputasi 

# Web server adalah sebuah perangkat lunak server yang berfungsimenerima permintaan HTTP atau HTTPS dari web client danmengirimkan jawaban dalam bentuk halaman-halaman web yangumumnya berbentuk dokumen HTML. Dokumen HTML yang dikirimmerupakan dokumen yang dibentuk dari bahasa markup yang plaintext.Artinya, dokumen berisi kode-kode HTML murni tanpa proses enkripsi.
# True Label: RPL, Predicted Label: komputasi 

# Berjalannya koneksi jaringan komputer dengan lancar dan tanpa gangguan merupakan harapan seluruh penggunanya,terutama pihak yang bertanggung jawab pada bidang itu, yakni administrator jaringan.
# True Label: RPL, Predicted Label: RPL 

Sistem  informasi  akademik  (SIAKAD) merupakan  sistem  informasi  yang  berfungsi  menangani pengelolaan  dan  penyajian  data-data  akademik,  yang  oleh pihak  fakultas  SIAKAD  dianggap  sangat  penting  dalam memberikan  pelayanan  mahasiswa  yang  membutuhkan informasi akademik.
True Label: RPL, Predicted Label: komputasi 

Web server adalah sebuah perangkat lunak server yang berfungsimenerima permintaan HTTP atau HTTPS dari web client danmengirimkan jawaban dalam bentuk halaman-halaman web yangumumnya berbentuk dokumen HTML. Dokumen HTML yang dikirimmerupakan dokumen yang dibentuk dari bahasa markup yang plaintext.Artinya, dokumen berisi kode-kode HTML murni tanpa proses enkripsi.
True Label: RPL, Predicted Label: komputasi 

Berjalannya koneksi jaringan komputer dengan lancar dan tanpa gangguan merupakan harapan seluruh penggunanya,terutama pihak yang bertanggung jawab pada bidang itu, yakni administrator jaringan.
True Label: RPL, Predicted Label: RPL 



result when use model distiluse-base-multilingual-cased-v2

## References


*   How To Classify Text With Python, Transformers & scikit-learn - [NewsCatcher](https://newscatcherapi.com/blog/how-to-classify-text-with-python-transformers-and-scikit-learn)
*   A Comprehensive Guide to Understand and Implement Text Classification in Python - [AnalyticsVidhya](https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/)
*   Sentence-BERT for spaCy [Spacy-Sentence-Bert](https://github.com/MartinoMensio/spacy-sentence-bert)
*   Sentence Transformers [see](https://www.sbert.net/index.html)
